# P3 OpenStreet Map Data with SQL

In [1]:
import sqlite3
import codecs
import csv
import pprint

def UnicodeDictReader(utf8_data, **kwargs):
    csv_reader = csv.DictReader(utf8_data, **kwargs)
    for row in csv_reader:
        yield {key: unicode(value, 'utf-8') for key, value in row.iteritems()}
        
# Create a dabase file
sqlite_file="HoustonSW_OSM.db"
db = sqlite3.connect(sqlite_file)
#Create a cursor object
c = db.cursor()
# Create the table, specifying the column names and data types:
c.execute('''
    CREATE TABLE nodes(
    nodeId INTEGER,
    lat REAL, 
    lon REAL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT,
    PRIMARY KEY(nodeId))
    ''')
c.execute('''
    CREATE TABLE nodes_tags(
    nodeId INTEGER,
    key TEXT,
    value TEXT, 
    type TEXT,
    FOREIGN KEY(nodeId) REFERENCES nodes(nodeId))
    ''')
c.execute('''
    CREATE TABLE ways(
    wayId INTEGER,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT,
    PRIMARY KEY(wayId))
    ''')
c.execute('''
    CREATE TABLE ways_nodes(
    wayId INTEGER,
    nodeId INTEGER,
    position INTEGER, 
    FOREIGN KEY(wayId) REFERENCES ways(wayId),
    FOREIGN KEY(nodeId) REFERENCES nodes(nodeId))
    ''')
c.execute('''
    CREATE TABLE ways_tags(
    wayId INTEGER,
    key TEXT,
    value TEXT, 
    type TEXT,
    FOREIGN KEY(wayId) REFERENCES ways(wayId))
    ''')

# commit the changes
db.commit()
# Read in the csv file as a dictionary, format the
# data as a list of tuples:
#-------------------------------------------------------------------------------
# Processing Nodes
#--------------------------------------------------------------------------------
with open('nodes.csv','rb') as fin:
    dr = UnicodeDictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['lat'],i['lon'], i['user'],
              i['uid'],i['version'], i['changeset'],i['timestamp']) for i in dr]    
# insert the formatted data
c.executemany('''
INSERT INTO nodes
 VALUES (?,?,?,?,?,?,?,?);
''', to_db)
#--------------------------------------------------------------------------------
# Processing Node Tags
#--------------------------------------------------------------------------------
with open('nodes_tags.csv','rb') as fin:
    dr = UnicodeDictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'],i['value'], i['type']) for i in dr]    
# insert the formatted data
c.executemany('''
INSERT INTO nodes_tags
 VALUES (?,?,?,?);
''', to_db)
#-------------------------------------------------------------------------------
# Processing ways
#--------------------------------------------------------------------------------
with open('ways.csv','rb') as fin:
    dr = UnicodeDictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['user'],i['uid'], i['version'],
              i['changeset'],i['timestamp']) for i in dr]    
# insert the formatted data
c.executemany('''
INSERT INTO ways
 VALUES (?,?,?,?,?,?);
''', to_db)
#--------------------------------------------------------------------------------
# Processing ways nodes
#--------------------------------------------------------------------------------
with open('ways_nodes.csv','rb') as fin:
    dr = UnicodeDictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['node_id'],i['position']) for i in dr]    
# insert the formatted data
c.executemany('''
INSERT INTO ways_nodes
 VALUES (?,?,?);
''', to_db)
#--------------------------------------------------------------------------------
# Processing way Tags
#--------------------------------------------------------------------------------
with open('ways_tags.csv','rb') as fin:
    dr = UnicodeDictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'],i['value'], i['type']) for i in dr]    
# insert the formatted data
c.executemany('''
INSERT INTO ways_tags
 VALUES (?,?,?,?);
''', to_db)
#---------------------------------------------------------------------------------
# commit the changes
db.commit()
# check
c.execute('SELECT * FROM nodes LIMIT 10')
all_rows = c.fetchall()
print all_rows
    
#close db
db.close()


[(151339751, 29.7705062, -95.4954973, u'davidearl', 3582, u'2', 177010, u'2008-12-06T18:17:40Z'), (151340813, 29.6638438, -95.6310571, u'davidearl', 3582, u'2', 177010, u'2008-12-06T18:17:45Z'), (151357671, 29.6680091, -95.4429963, u'davidearl', 3582, u'2', 177010, u'2008-12-06T18:18:47Z'), (151365266, 29.749123, -95.5721842, u'scottyc', 496606, u'4', 14675589, u'2013-01-16T18:17:48Z'), (151365267, 29.749119, -95.573012, u'woodpeck_fixbot', 147510, u'2', 3486233, u'2009-12-29T23:15:28Z'), (151365269, 29.748218, -95.572979, u'woodpeck_fixbot', 147510, u'2', 3486233, u'2009-12-29T23:15:28Z'), (151365270, 29.748182, -95.573085, u'woodpeck_fixbot', 147510, u'2', 3486233, u'2009-12-29T23:15:29Z'), (151365271, 29.748163, -95.573219, u'woodpeck_fixbot', 147510, u'2', 3486233, u'2009-12-29T23:15:29Z'), (151365273, 29.7478777, -95.5732226, u'afdreher', 1110270, u'5', 29148052, u'2015-02-28T01:59:18Z'), (151365344, 29.641142, -95.485489, u'woodpeck_fixbot', 147510, u'2', 3486233, u'2009-12-29T23

# Perform SQL queries

In [1]:
import sqlite3
import pprint
sqlite_file="HoustonSW_OSM.db"
db = sqlite3.connect(sqlite_file)
c = db.cursor()

## Number of Nodes

In [2]:
c.execute('''
SELECT COUNT(*) FROM nodes;
''')
all_rows = c.fetchall()
print 'number of nodes: '
pprint.pprint(all_rows)

number of nodes: 
[(285952,)]


## Number of Ways

In [3]:
c.execute('''
SELECT COUNT(*) FROM ways;
''')
all_rows = c.fetchall()
print 'number of ways: ', all_rows

number of ways:  [(57639,)]


## Number of unique users

In [4]:
c.execute('''
SELECT COUNT(DISTINCT(u.uid)) 
FROM (SELECT uid FROM nodes
UNION ALL SELECT uid FROM ways) as u;
''')
all_rows = c.fetchall()
print 'number of unique users: ', all_rows

number of unique users:  [(427,)]


## Number of Fast Food Restaurants

In [7]:
c.execute('''
SELECT COUNT(*)
FROM nodes_tags
WHERE key='amenity' 
AND value='fast_food';
''')
all_rows = c.fetchall()
print 'number of fast_food restaurants: ', all_rows

number of fast_food restaurants:  [(121,)]


## Nodes that are used

In [23]:
c.execute('''
SELECT COUNT(DISTINCT(used_nodes.nodeId)) as num_un
FROM (SELECT nodeId from nodes_tags 
UNION ALL SELECT nodeId FROM ways_nodes) as used_nodes;
''')
all_rows = c.fetchall()
print 'number of used nodes: ', all_rows

number of used nodes:  [(285946,)]


# Nodes that are shared by nodes_tags & way_nodes

In [27]:
c.execute('''
SELECT COUNT(DISTINCT(nodeId))
FROM nodes_tags
WHERE nodeId in (SELECT DISTINCT(nodeId) FROM ways_nodes);
''')
all_rows = c.fetchall()
print 'number of shared node ', all_rows

number of shared node  [(8483,)]


In [3]:
c.execute('''
SELECT nodes_tags.value, COUNT(DISTINCT(nodeId)) as num
FROM nodes_tags
WHERE nodeId in (SELECT DISTINCT(nodeId) FROM ways_nodes)
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
''')
all_rows = c.fetchall()
print 'shared nodes '
pprint.pprint(all_rows)

shared nodes 
[(u'traffic_signals', 1996),
 (u'turning_circle', 1663),
 (u'tower', 1618),
 (u'crossing', 885),
 (u'pole', 742),
 (u'yes', 519),
 (u'gate', 392),
 (u'level_crossing', 285),
 (u'uncontrolled', 215),
 (u'motorway_junction', 168)]


In [4]:
# This part of the code take a long time to finish
c.execute('''
SELECT ways_tags.value, COUNT (ways_nodes.nodeId) as num
FROM (ways_nodes JOIN nodes_tags
ON ways_nodes.nodeId=nodes_tags.nodeId)
JOIN ways_tags 
ON (ways_tags.wayId=ways_nodes.wayId)
WHERE nodes_tags.value=="traffic_signals" AND
    ways_tags.key="name"
GROUP BY ways_tags.value
ORDER BY num DESC
LIMIT 10;
''')
all_rows = c.fetchall()
print 'Roads with most traffic Lights '
pprint.pprint(all_rows)

Roads with most traffic Lights 
[(u'Westheimer Road', 198),
 (u'Richmond Avenue', 181),
 (u'Bellaire Boulevard', 147),
 (u'Bissonnet Street', 146),
 (u'Beechnut Street', 144),
 (u'Fondren Road', 100),
 (u'South Braeswood Boulevard', 98),
 (u'Southwest Freeway Frontage Road', 96),
 (u'Chimney Rock Road', 91),
 (u'Westpark Drive', 88)]
